**4.	Trends für verschiedenen Kryptowährungen vergleichen, Ereignisse (z.B politische Ereignisse)**

In [11]:
from dotenv import load_dotenv
import requests
from api_key import API_KEY
import numpy as np
import pandas as pd
from openai import OpenAI
import os
import json
import textwrap
import functions as f


In [7]:
load_dotenv()

True

In [8]:
# The role for openai prompt
role = """Du bist ein Krypto-Marktanalyse-Bot.

Antwortsprache: **Deutsch**.

Aufgabe:
1. Suche mithilfe von Web-Recherche nach relevanten Ereignissen, die den genannten Kurs­sprung/-fall erklären könnten.
2. Gib **exakt** folgendes JSON-Array zurück (keine Kommentare, kein Markdown):

[
  {
    "zeitraum": "<TT.MM.JJJJ-TT.MM.JJJJ>",
    "preisänderung": "<+12.34 %>",
    "ursache": "<max. 2 Sätze, keine URLs>",
    "ereignistyp": "<Politik | Markt | Regulierung | Meme/Trend | Unbekannt>",
    "vertrauen": "<Hoch | Mittel | Gering>",
    "quellen": ["<mind. 1, max. 5 valide URLs>"]
  }
]

Richtlinien:
- Nenne **mindestens eine** Quelle; falls keine verlässliche Quelle gefunden, setze leeres Array und `vertrauen="Gering"`.
- Füge **keine** URL in `ursache` ein.
- Nutze Social-Media-Erwähnungen (X/Twitter, Reddit) für Meme-Coins, kennzeichne das als `ereignistyp="Meme/Trend"`."""


In [ ]:
def biggest_moves(prices, window=5, n_results=None, direction="both", allow_overlap=False):
    """
    Die Funktion liefert die stärksten prozentualen Kursbewegungen (auf- oder abwärts) 
    eines Tokens über rollierende Zeitfenster.

    • window: Länge des Fensters in Tagen/Zeilen  
    • direction: 'up', 'down' oder 'both'  
    • n_results: maximale Zahl der zurückgegebenen, nicht überlappenden Fenster  
    • allow_overlap=False verhindert, dass sich Zeiträume überschneiden

    Rückgabe: DataFrame mit Start- und Enddatum, Preisen, %-Änderung und Richtung.
    """
    if "price" not in prices:
        raise ValueError("prices DataFrame must contain column 'price'")

    pct_change = prices["price"].pct_change(periods=window) * 100 # Prozentuale Änderung über das Fenster

    df = pd.DataFrame({
        "end_price": prices["price"], # Aktueller Preis am Ende des Fensters
        "pct_change": pct_change # Prozentuale Änderung
    }).dropna() # Entferne Zeilen mit NaN-Werten

    df["start_price"] = df["end_price"].shift(window) # Preis am Anfang des Fensters
    df["start"] = df.index - pd.Timedelta(days=window) # Startdatum des Fensters
    df["end"] = df.index # Enddatum des Fensters
    df["direction"] = np.where(df["pct_change"] > 0, "up", "down") # Richtung der Bewegung
    df["abs_move"] = df["pct_change"].abs() # Absolute Bewegung in Prozent

    if direction in {"up", "down"}: # Filter nach Richtung
        df = df[df["direction"] == direction]

    ranked = df.sort_values("abs_move", ascending=False) # Sortiere nach absoluter Bewegung

    if not allow_overlap: # Verhindere überlappende Zeiträume
        kept_idx, spans = [], [] # Liste für die Indizes der behaltenen Zeiträume und deren Spannen
        for idx, row in ranked.iterrows():
            s, e = row["start"], row["end"] # Start- und Enddatum des Fensters
            if not any((s <= ee) and (e >= ss) for ss, ee in spans): # Überprüfe, ob der Zeitraum überlappt
                kept_idx.append(idx) # Füge den Index hinzu, wenn kein Überlappen vorliegt
                spans.append((s, e)) # Speichere die Spanne des Zeitraums
            if n_results is not None and len(kept_idx) == n_results: # Wenn die maximale Anzahl an Ergebnissen erreicht ist, beende die Schleife
                break
        ranked = ranked.loc[kept_idx] # Behalte nur die Indizes der nicht überlappenden Zeiträume

    result = ( # Erstelle das Ergebnis-DataFrame
        ranked[["start", "end", "start_price", "end_price", "pct_change", "direction"]]
        .sort_values("pct_change", ascending=(direction == "down")) # Sortiere nach prozentualer Änderung
        .head(n_results)
        .reset_index(drop=True)
        .round({"start_price": 2, "end_price": 2, "pct_change": 2})
    )

    result["start"] = pd.to_datetime(result["start"]).dt.date # Konvertiere Startdatum zu Datum
    result["end"] = pd.to_datetime(result["end"]).dt.date # Konvertiere Enddatum zu Datum
    
    return result

In [ ]:
def load_news(token, start, end, pct_change):
  """ Die Funktion ruft openai api auf und rescheschiert, ob die Kursbewegungen mit globalen Ereignissen zur Verbindung stehen.
  Und Wenn ja, aus welchen Gründen Tokenpreis an- oder abgestiegen ist. """

  client = OpenAI(
      api_key = os.getenv('OPENAI_API_KEY') # OpenAI API Key aus der .env Datei
      )

  message = f'{token}, period: from  {start} to {end}, returns: {pct_change}' # Nachricht, die an das OpenAI Modell gesendet wird

  response = client.chat.completions.create(
    model="gpt-4o-search-preview",
    messages=[
      {
        "role": "system", # Systemnachricht, die die Rolle des Modells definiert
        "content": [ # Inhalt der Systemnachricht
          {
            "text": role, # Die Rolle des Modells, die die Aufgabe und Richtlinien definiert
            "type": "text"
          }
        ]
      },
      {
        "role": "user", # Benutzernachricht, die die Anfrage enthält
        "content": [ # Inhalt der Benutzernachricht
          {
            "type": "text",
            "text": message # Die Nachricht, die an das Modell gesendet wird
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    web_search_options={
      "user_location": {
        "type": "approximate",
        "approximate": {
          "country": "DE"
        }
      }
    }
  )
  return response.choices[0].message.content # Antwort des Modells, die die Ergebnisse der Websuche enthält

In [15]:
token = 'solana'

prices = f.parse_token_data(token, 365) # Lade die Tokenpreise für die letzten 365 Tage
result = biggest_moves(prices, window=7, n_results=3) # Finde die größten Kursbewegungen über 7-Tage-Fenster
print(result)

# Suche nach Ereignissen, die die Kursbewegungen erklären:
json_str = load_news(token=token, start=result['start'], end= result['end'], pct_change=result['pct_change'])
events = json.loads(json_str)  # JSON-String in eine Python-Liste von Dictionaries umwandeln
print(events)

# Ausgabe der Ereignisse in einem lesbaren Format:
for event in events:
    print(f"\nZeitraum: {event['zeitraum']}")
    print(f"Preisänderung: {event['preisänderung']}")
    print(textwrap.fill(event['ursache'], width=80))
    print(f"Ereignistyp: {event['ereignistyp']}")
    print(f"Vertrauen: {event['vertrauen']}")
    print("Quellen:")
    if event['quellen']: 
        for url in event['quellen']:
            print(f"  - {url}")
    else: print('Keine Quellen')
    print()

        start         end  start_price  end_price  pct_change direction
0  2024-11-05  2024-11-12       157.61     222.64       41.26        up
1  2025-01-12  2025-01-19       188.06     262.56       39.61        up
2  2024-07-30  2024-08-06       182.54     129.70      -28.95      down
[{'zeitraum': '05.11.2024-12.11.2024', 'preisänderung': '+41,26 %', 'ursache': 'Nach den US-Wahlen am 5. November 2024 stieg der Solana-Kurs aufgrund der Erwartung kryptofreundlicherer Regulierung und eines allgemeinen Marktoptimismus.', 'ereignistyp': 'Politik', 'vertrauen': 'Hoch', 'quellen': ['https://www.crypto-insiders.de/nachrichten/solana/drei-gruende-warum-solana-kurz-vor-neuem-rekord-steht/']}, {'zeitraum': '12.01.2025-19.01.2025', 'preisänderung': '+39,61 %', 'ursache': 'Die Ankündigung von Donald Trump, eine strategische Kryptoreserve für die USA einzurichten, die Solana einschließt, führte zu einem starken Kursanstieg.', 'ereignistyp': 'Politik', 'vertrauen': 'Hoch', 'quellen': ['https://bit